## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-20-18-53-34 +0000,nyt,"Saudi Arabia’s Prince Has Big Plans, but His G...",https://www.nytimes.com/2025/11/19/business/pi...
1,2025-11-20-18-52-00 +0000,wsj,U.S. job growth defied expectations in September,https://www.wsj.com/economy/central-banking/de...
2,2025-11-20-18-49-56 +0000,nypost,"Dick Cheney funeral draws big Dem names Biden,...",https://nypost.com/2025/11/20/us-news/dick-che...
3,2025-11-20-18-49-00 +0000,wsj,UPS Plane-Crash Probe Identifies ‘Fatigue Crac...,https://www.wsj.com/business/airlines/ups-plan...
4,2025-11-20-18-49-00 +0000,wsj,Energy Department Cuts Two Major Clean-Energy ...,https://www.wsj.com/politics/policy/energy-dep...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,43
298,new,18
81,will,15
200,house,15
779,epstein,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
273,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...,119
287,2025-11-20-01-41-44 +0000,wapo,U.S. pushes new Ukraine peace plan as Trump’s ...,https://www.washingtonpost.com/national-securi...,106
274,2025-11-20-02-32-00 +0000,wsj,Trump Says He Will Meet With Mamdani Friday at...,https://www.wsj.com/politics/elections/trump-s...,100
282,2025-11-20-02-14-00 +0000,wsj,Trump Administration Pushes New Plan for Endin...,https://www.wsj.com/world/trump-administration...,97
269,2025-11-20-02-48-43 +0000,nyt,Trump and Mamdani Will Meet at White House on ...,https://www.nytimes.com/2025/11/19/nyregion/tr...,95


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
273,119,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...
261,82,2025-11-20-03-35-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
340,65,2025-11-19-21-52-57 +0000,nypost,‘Off the charts’ demand for AI chips powers st...,https://nypost.com/2025/11/19/business/nvidia-...
34,64,2025-11-20-18-05-00 +0000,wsj,Former President George W. Bush hailed Dick Ch...,https://www.wsj.com/politics/vice-president-di...
64,57,2025-11-20-17-20-17 +0000,nypost,Rubio casts doubt on pro-Russian Ukraine peac...,https://nypost.com/2025/11/20/world-news/rubio...
147,43,2025-11-20-13-19-42 +0000,nyt,Delayed Jobs Data to Give Snapshot of a Labor ...,https://www.nytimes.com/live/2025/business/job...
40,43,2025-11-20-17-55-23 +0000,nypost,Israeli strikes kill 27 in Gaza after alleged ...,https://nypost.com/2025/11/20/world-news/israe...
99,41,2025-11-20-15-59-56 +0000,nypost,"Ex-girlfriend, new beau teamed up to murder fo...",https://nypost.com/2025/11/20/us-news/woman-ch...
142,37,2025-11-20-13-42-14 +0000,nypost,Slain Carnival cruise teen Anna Kepner’s broth...,https://nypost.com/2025/11/20/us-news/slain-ca...
63,36,2025-11-20-17-20-47 +0000,nypost,Kamala Harris and Jill Biden come face-to-face...,https://nypost.com/2025/11/20/us-news/kamala-h...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
